In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path

from PyEMD import CEEMDAN

tqdm.pandas()

## Notes
The final program will likely be a CLI with arguments, some of the options should be:
- Input data
- Output data
- Whether to interpolate over missing data (initially linear)
- Whether to reject components which are primarily noise (with note that this may not be valid with CEEMDAN)
    - Apriori or aposteriori test? 
- Tolerance when matching components by frequency
- Whether to allow input components to be used in multiple output components
- Time range for training/testing/prediction data
- Noise values

In [ ]:
folder = Path("data") / "run_notebook"

In [ ]:
# Load data
# TODO - note that we're not interpolating over gaps here
from pySPADS.processing.bridge import load_shorecast, load_hindcast, load_SLP

shore_df = load_shorecast()
hind_df = load_hindcast()
pca_df = load_SLP()

In [ ]:
# Interpolate over NaNs in data
def interpolate(df):
    trange = list(range(df.index.min(), df.index.max() + 1))
    return df.reindex(trange, fill_value=np.nan).interpolate(method="linear")


shore_df = interpolate(shore_df)
hind_df = interpolate(hind_df)
pca_df = interpolate(pca_df)

In [ ]:
from pySPADS.visualisation import paper

f = paper.fig1(
    pca_df["PC0"],
    hind_df["Hs"],
    hind_df["Tp"],
    hind_df["Dir"],
    "2004-01-01",
    "2016-01-01",
)

f.savefig(folder / "figures" / "fig1.png")

In [ ]:
# Set up a list of each timeseries to decompose
# Each item in the list is a tuple of:
#   label, source df, source column
series = (
    [
        ("output", shore_df, "y"),
    ]
    + [(label, hind_df, label) for label in hind_df.columns]
    + [(label, pca_df, label) for label in pca_df.columns]
)

In [ ]:
# Set up CEEMD driver
NR = 100
ns = np.arange(0.1, 0.5, 0.05)

In [ ]:
# Individually decompose each signal
imf_folder = folder / "imfs"
os.makedirs(imf_folder, exist_ok=True)

for label, df, col in series:
    file = f"{imf_folder}/{label}_{25}.csv"
    if os.path.exists(file):
        print(f"Skipping {label}")
    else:
        print(f"Decomposing {label}")
        ceemd = CEEMDAN(trails=NR, epsilon=0.2, processes=8, parallel=True)
        imfs = ceemd.ceemdan(df[col].to_numpy(), df.index.to_numpy(), progress=True)
        imfs_df = pd.DataFrame(imfs.T, index=df.index)
        imfs_df.to_csv(file)

In [ ]:
# Load all the IMFs
all_imfs = {}
for label, _, _ in series:
    file = f"{imf_folder}/{label}_{25}.csv"
    all_imfs[label] = pd.read_csv(file, index_col=0)

    # Column names are strings, convert to ints
    all_imfs[label].columns = all_imfs[label].columns.astype(int)

In [ ]:
# Plot the IMFs
from pySPADS.visualisation.imfs import plot_imfs

imf_plot_folder = folder / "figures" / "imfs"
os.makedirs(folder, exist_ok=True)

for label, imf_df in tqdm(all_imfs.items(), desc="Plotting IMFs"):
    plot_imfs(imf_df.to_numpy().T, label, imf_plot_folder / f"{label}.png")

In [ ]:
## Reject components which are primarily noise
# Note that this might not be valid when applied to the result of CEEMDAN (rather than EMD), so it should be optional
from PyEMD.checks import whitenoise_check

for label, imf_df in all_imfs.items():
    print(f"Checking {label}")
    sig = whitenoise_check(imf_df.to_numpy().T)  # , test_name='apriori')
    rejects = [k for k, v in sig.items() if v == 0]
    print(f"Rejecting: {rejects}")
    all_imfs[label] = imf_df.drop(columns=[i - 1 for i in rejects])

In [ ]:
# Number of components in each signal
print("Number of component IMFs in each signal:")
for label, imf_df in all_imfs.items():
    print(f"{label:>12}: {imf_df.shape[1]}")

In [ ]:
## Match up components in input/output by frequency
from pySPADS.processing.recomposition import component_frequencies

# Get the maxima of each component
max_imfs = max([imf_df.shape[1] for imf_df in all_imfs.values()])
freq_df = pd.DataFrame(columns=all_imfs.keys(), index=list(range(max_imfs)))
for label, imf_df in all_imfs.items():
    freq_df[label] = component_frequencies(imf_df)

freq_df

In [ ]:
fig2 = paper.fig2(shore_df["y"], all_imfs["output"], "1999-01-01", "2017-01-01")
fig2.savefig(folder / "figures" / "fig2.png")

In [ ]:
fig3 = paper.fig3(all_imfs, "output", "1999-01-01", "2017-01-01")
fig3.savefig(folder / "figures" / "fig3.png")

In [ ]:
input_cols = set(freq_df.columns) - {"output"}
output_index = all_imfs["output"].columns.tolist()

In [ ]:
# Find the closest match for each component in output
from pySPADS.processing.recomposition import nearest_frequency

nearest_freq = nearest_frequency(freq_df["output"], freq_df.drop(columns=["output"]))

nearest_freq

In [ ]:
# Check to see if difference in frequencyes is within tolerance
from pySPADS.processing.recomposition import frequency_difference

diff_df = frequency_difference(
    freq_df["output"], freq_df.drop(columns=["output"]), nearest_freq
)
diff_df

In [ ]:
from pySPADS.processing.recomposition import relative_frequency_difference

rel_diff_df = relative_frequency_difference(
    freq_df["output"], freq_df.drop(columns=["output"]), nearest_freq
)
rel_diff_df

In [ ]:
tolerance = 0.25

In [ ]:
# Check that each output component has some valid inputs
valid_components = (rel_diff_df < tolerance).sum(axis=1)
if any(valid_components == 0):
    raise ValueError(
        f"No valid input components for output components: {valid_components[valid_components == 0].index}"
    )

if any(valid_components < 3):
    print(
        f"Warning: some output components have less than 3 valid input components: {valid_components[valid_components < 3].index}"
    )

In [ ]:
rel_diff_df < tolerance

In [ ]:
# Show the components which are used for each output component
print("Components used for each output component:")
for i in output_index:
    print(f"{i:>5} : {np.sum(rel_diff_df.loc[i] < tolerance)}")

In [ ]:
# TODO - set nearest_freq to NaN for components which are not used

In [ ]:
# TODO - note below - we have more output data than input, we should have truncated the output data before fitting!

In [ ]:
## Linear regression of components to output signal
# Note that this is much simpler if the original data interpolated over NaNs
from sklearn.linear_model import LinearRegression
from pySPADS.optimization import MReg2

# TODO - note below - we have more output data than input, we should have truncated the output data before fitting!
hindcast_index = all_imfs["output"].index[
    all_imfs["output"].index.isin(all_imfs["PC0"].index)
]

imf_predictions = pd.DataFrame(index=hindcast_index, columns=output_index)
mreg_predictions = pd.DataFrame(index=hindcast_index, columns=output_index)

for i, imf in enumerate(output_index):
    print(f"Fitting component {imf}")
    X = pd.DataFrame(index=hindcast_index)
    for label in input_cols:
        if rel_diff_df.loc[imf, label] < tolerance:
            # X[label] = all_imfs[label].iloc[:, nearest_freq.loc[imf, label]]
            X[label] = all_imfs[label].loc[
                all_imfs[label].index.isin(hindcast_index), nearest_freq.loc[imf, label]
            ]
    y = all_imfs["output"].loc[hindcast_index, imf]
    reg = LinearRegression().fit(X, y)
    print(f"R^2: {reg.score(X, y)}")
    print(f"Coefficients: {reg.coef_}")
    mreg2 = MReg2().fit(X, y)
    coefs = mreg2.coef_
    print(coefs)
    print(f"Intercept: {reg.intercept_}")
    # Plot the fit
    plt.figure()
    plt.plot(y, label="output")
    plt.plot(y.index, reg.predict(X), label="fit")
    mreg_pred = np.sum([X[c] * coefs[i] for i, c in enumerate(X.columns)], axis=0)
    plt.plot(y.index, mreg_pred, label="mreg")
    imf_predictions.loc[:, imf] = reg.predict(X)
    mreg_predictions.loc[:, imf] = mreg_pred
    plt.legend()
    plt.title(f"Component {imf}")
    plt.show()
    print("\n\n")

In [ ]:
reg.coef_

In [ ]:
mreg_pred = np.sum([X[c] * coefs[i] for i, c in enumerate(X.columns)], axis=0)

In [ ]:
all_imfs["output"].sum(axis=1)

In [ ]:
imf_predictions.sum(axis=1)

In [ ]:
fig4 = paper.fig4(
    shore_df["y"], imf_predictions.sum(axis=1), "2010-01-01", "2017-01-01"
)

fig4.savefig(folder / "figures" / "fig4.png")

In [ ]:
fig4a = paper.fig4(
    shore_df["y"], mreg_predictions.sum(axis=1), "2010-01-01", "2017-01-01"
)

fig4a.savefig(folder / "figures" / "fig4a.png")

In [ ]:
from pySPADS.processing.bridge import datenum_to_datetime

# TODO - note that above covers 1998-, paper uses only 2010- data
datenum_to_datetime(737000)

In [ ]:
# TODO:
#   repeat the process for other noise values (and average?)
#   tweak tolerance, noises, other settings?
#   check regression implementation
#